In [22]:
import os
# Change to the parent directory
os.chdir('..')
import torch
import torchvision
import os
from os.path import join as j_
from PIL import Image
import pandas as pd
import numpy as np

# loading all packages here to start
from panderm_model import get_encoder
from panderm_model.downstream.extract_features import extract_features_from_dataloader
from panderm_model.downstream.eval_features.linear_probe import eval_linear_probe
from panderm_model.downstream.eval_features.metrics import get_eval_metrics, print_metrics

from datasets.derm_data import Derm_Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Downloading Model weights + Creating Model

Set model_name to one of 'PanDerm' 'SwAVDerm'  'dinov2' 'imgnet_large21k', which is our model and the main comparative models.

In [23]:
from models import get_encoder
model_name='PanDerm'
model, eval_transform = get_encoder(model_name)
_ = model.eval()

loading model checkpoint
VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elemen

### Prepare Evaluation Dataset (HAM10000 as an example) and Important Hyperparameters

#### Dataset Setup
1. Download the provided [dataset](https://drive.google.com/file/d/1D9Q4B50Z5tyj5fd5EE9QWmFrg66vGvfA/view).
2. Update the following key paths:
   - `csv_path`: Path to the CSV file containing data paths and labels
   - `root_path`: Directory containing the image files
   - `output_dir`: Directory where results will be saved

#### Key Parameters
Adjust these important parameters:

- `nb_classes`: Set this to the number of classes in your evaluation dataset
- `batch_size`: Adjust based on the memory size of your GPU
- `percent_data`: Controls the percentage of training data used
  - Example: 0.1 means evaluate models using 10% training data
  - Modify this for label efficiency generalization experimentseriments.


In [24]:
csv_path="/home/share/Uni_Eval/HAM10000_clean/ISIC2018_splits/HAM_clean.csv" 
root_path="/home/share/Uni_Eval/HAM10000_clean/ISIC2018/"
output_dir='/home/share/FM_Code/PanDerm/LP_Eval/output_dir/Task1/'+model_name
# set the class number of your evaluation dataset
nb_classes=7
model = model.to(device)
if nb_classes == 2:
    binary = True
else:
    binary = False

batch_size=1000
num_workers=8
percent_data=1.0


### Prepare dataloader

In [25]:
df = pd.read_csv(csv_path)
dataset_train = Derm_Dataset(df=df,
                            root=root_path,
                            train=True,
                            transforms=eval_transform,
                            binary=binary,
                            data_percent=percent_data)
dataset_val = Derm_Dataset(df=df,
                          root=root_path,
                          val=True,
                          transforms=eval_transform,
                          binary=binary)
dataset_test = Derm_Dataset(df=df,
                           root=root_path,
                           test=True,
                           transforms=eval_transform,
                           binary=binary)
print('train size:', len(dataset_train), ',val size:', len(dataset_val), ',test size:', len(dataset_test))


import time
from panderm_model.downstream.extract_features import extract_features_from_dataloader

train_dataloader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

val_dataloader = torch.utils.data.DataLoader(
    dataset_val,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)
test_dataloader = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

train size: 8207 ,val size: 575 ,test size: 1232


### Feature Extraction

In [26]:
start = time.time()
# extract features from the train and test datasets (returns dictionary of embeddings and labels)
train_features = extract_features_from_dataloader(model, train_dataloader)
val_features = extract_features_from_dataloader(model, val_dataloader)
test_features = extract_features_from_dataloader(model, test_dataloader)

# convert these to torch
train_feats = torch.Tensor(train_features['embeddings'])
train_labels = torch.Tensor(train_features['labels']).type(torch.long)
val_feats = torch.Tensor(val_features['embeddings'])
val_labels = torch.Tensor(val_features['labels']).type(torch.long)
test_feats = torch.Tensor(test_features['embeddings'])
test_labels = torch.Tensor(test_features['labels']).type(torch.long)
test_filenames = test_features['filenames']
elapsed = time.time() - start
print(f'Took {elapsed:.03f} seconds')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:26<00:00, 13.10s/it]

Took 119.596 seconds


### Linear Probe Evaluation.

In [27]:
"""linear evaluation"""
from panderm_model.downstream.eval_features.linear_probe import eval_linear_probe
dataset_name=str(csv_path).split('/')[-1].split('.')[0]
print('Evaluation Dataset:',dataset_name)
linprobe_eval_metrics, linprobe_dump = eval_linear_probe(
    train_feats=train_feats,
    train_labels=train_labels,
    valid_feats=None,
    valid_labels=val_labels,
    test_feats=test_feats,
    test_labels=test_labels,
    test_filenames=test_filenames,
    max_iter=1000,
    verbose=True, seed=0,
    out_dir=output_dir,
    dataset_name=dataset_name
)

print_metrics(linprobe_eval_metrics)

Evaluation Dataset: HAM_clean
Linear Probe Evaluation: Train shape torch.Size([8207, 1024])
Linear Probe Evaluation: Test shape torch.Size([1232, 1024])
NUM_C, Cost: 7 71.68
Linear Probe Evaluation (Train Time): Best cost = 71.680
Linear Probe Evaluation (Train Time): Using only train set for training. Train Shape:  torch.Size([8207, 1024])
0
(Before Training) Loss: 2.023
(After Training) Loss: 0.381
Linear Probe Evaluation (Test Time): Test Shape torch.Size([1232, 1024])
Classification report:
              precision    recall  f1-score   support

           0       0.78      0.60      0.68        35
           1       0.68      0.82      0.74        44
           2       0.82      0.79      0.81       107
           3       0.88      0.88      0.88         8
           4       0.70      0.63      0.66        70
           5       0.97      0.98      0.97       951
           6       1.00      0.88      0.94        17

    accuracy                           0.92      1232
   macro avg

<Figure size 720x576 with 0 Axes>